In [504]:
import pandas as pd

In [505]:
import numpy as np

In [506]:
dataset1 = pd.read_csv('owid-covid-data.csv')
dataset = dataset1.copy()

In [507]:
dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,new_vaccinations,total_vaccinations_per_hundred,new_vaccinations_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [508]:
dataset['new_cases']= dataset['new_cases'].replace(np.nan,0)

In [509]:
dataset['total_cases']= dataset['total_cases'].replace(np.nan,0)

In [510]:
#Get the value of the first case in each country
country_first_case = {}
country_first_case_date = {}
for i in dataset.location.unique():
    data = dataset[(dataset.location)==i]
    country_first_case[i]=0
    
    #Reset the index
    data['index']=[i for i in range(len(data))]
    data = data.set_index('index')
    
    for j in range(len(data)):
        if data.loc[j,'new_cases']!=0:
            First_case = data.loc[j,'new_cases']
            country_first_case[i]=First_case
            country_first_case_date[i]=data.loc[j,'date']
            break

<ipython-input-510-46063f1fcb74>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=[i for i in range(len(data))]


In [511]:
#Get the value of the last case in each country
country_last_case = {}
country_last_case_date = {}

for i in dataset.location.unique():
    data = dataset[(dataset.location)==i]
    
    #Reset the index
    data['index']=[i for i in range(len(data))]
    data = data.set_index('index')
    
    for j in range(len(data)):
        if data.loc[j,'date']==np.max(data.date):
            Last_case = data.loc[j,'new_cases']
            country_last_case[i]=Last_case
            country_last_case_date[i]=data.loc[j,'date']
            break

<ipython-input-511-a85d041792c8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=[i for i in range(len(data))]


In [512]:
#Get the max and min % difference of new cases everyday in every country
country_percent_difference_max = {}
country_percent_difference_max_date = {}
country_percent_difference_max_value_1 = {}
country_percent_difference_max_value_2 = {}

country_percent_difference_min = {}
country_percent_difference_min_date = {}
country_percent_difference_min_value_1 = {}
country_percent_difference_min_value_2 = {}

for i in dataset.location.unique():
    data = dataset[(dataset.location)==i]
    
    #Reset the index
    data['index']=[i for i in range(len(data))]
    data = data.set_index('index')
    
    max = 0
    min = 0
    
    for j in range(len(data)-1):
        if (data.loc[j,'new_cases'])!=0:
            percent_difference = (data.loc[j+1,'new_cases']-data.loc[j,'new_cases']) / (data.loc[j,'new_cases']) 
            if percent_difference > max:
                max = percent_difference
                country_percent_difference_max_value_1[i] = data.loc[j,'new_cases']
                country_percent_difference_max_value_2[i] = data.loc[j+1,'new_cases']
                
                country_percent_difference_max_date[i]=data.loc[j,'date']
                
            if percent_difference < min:
                min = percent_difference
                country_percent_difference_min_value_1[i] = data.loc[j,'new_cases']
                country_percent_difference_min_value_2[i] = data.loc[j+1,'new_cases']
                
                country_percent_difference_min_date[i]=data.loc[j,'date']
            
    country_percent_difference_max[i]= max
    country_percent_difference_min[i]= min

<ipython-input-512-5aa13cde8f1c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=[i for i in range(len(data))]


In [513]:
#Extract month from date
dataset.date = pd.to_datetime(dataset.date)
dataset['month'] = pd.DatetimeIndex(dataset.date).month
dataset['year'] = pd.DatetimeIndex(dataset.date).year

In [514]:
#Get the max and min % difference of new cases everyday in every country
country_new_cases_jan_2020 = {}
country_new_cases_feb_2020 = {}
country_new_cases_mar_2020 = {}
country_new_cases_apr_2020 = {}
country_new_cases_may_2020 = {}
country_new_cases_june_2020 = {}
country_new_cases_july_2020 = {}
country_new_cases_aug_2020 = {}
country_new_cases_sep_2020 = {}
country_new_cases_oct_2020 = {}
country_new_cases_nov_2020 = {}
country_new_cases_dec_2020 = {}

for i in dataset.location.unique():
    data = dataset[(dataset.location)==i]
    data = data[(data.year)==2020]
    
    country_new_cases_jan_2020[i] = 0
    country_new_cases_feb_2020[i] = 0
    country_new_cases_mar_2020[i] = 0
    country_new_cases_apr_2020[i] = 0
    country_new_cases_may_2020[i] = 0
    country_new_cases_june_2020[i] = 0
    country_new_cases_july_2020[i] = 0
    country_new_cases_aug_2020[i] = 0
    country_new_cases_sep_2020[i] = 0
    country_new_cases_oct_2020[i] = 0
    country_new_cases_nov_2020[i] = 0
    country_new_cases_dec_2020[i] = 0
    
    
    for num_month in data.month.unique():
        data1 = data[(data.month)==num_month]
        #Reset the index
        data1['index']=[i for i in range(len(data1))]
        data1 = data1.set_index('index')

        sum_new_cases = 0
        for j in range(len(data1)):
            sum_new_cases += data1.loc[j,'new_cases']
            
        if num_month==1:
            country_new_cases_jan_2020[i]=sum_new_cases
        elif num_month==2:
            country_new_cases_feb_2020[i]=sum_new_cases
        elif num_month==3:
            country_new_cases_mar_2020[i]=sum_new_cases
        elif num_month==4:
            country_new_cases_apr_2020[i]=sum_new_cases
        elif num_month==5:
            country_new_cases_may_2020[i]=sum_new_cases
        elif num_month==6:
            country_new_cases_june_2020[i]=sum_new_cases
        elif num_month==7:
            country_new_cases_july_2020[i]=sum_new_cases
        elif num_month==8:
            country_new_cases_aug_2020[i]=sum_new_cases
        elif num_month==9:
            country_new_cases_sep_2020[i]=sum_new_cases
        elif num_month==10:
            country_new_cases_oct_2020[i]=sum_new_cases
        elif num_month==11:
            country_new_cases_nov_2020[i]=sum_new_cases
        elif num_month==12:
            country_new_cases_dec_2020[i]=sum_new_cases
                

<ipython-input-514-ec0c86ff8cf5>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['index']=[i for i in range(len(data1))]


In [515]:
#Get total case of the country
country_last_total_case = {}

for i in dataset.location.unique():
    data = dataset[(dataset.location)==i]
    
    #Reset the index
    data['index']=[i for i in range(len(data))]
    data = data.set_index('index')
    
    for j in range(len(data)):
        if data.loc[j,'date']==np.max(data.date):
            Last_case = data.loc[j,'total_cases']
            country_last_total_case[i]=Last_case
            break

<ipython-input-515-0d7580878501>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=[i for i in range(len(data))]


In [516]:
df = pd.DataFrame({'country_last_total_case':country_last_total_case,'country_first_case':country_first_case,'country_first_case_date':country_first_case_date,'country_last_case':country_last_case,
            'country_last_case_date':country_last_case_date, 'country_percent_difference_max':country_percent_difference_max,
             'country_percent_difference_max_date':country_percent_difference_max_date,'country_percent_difference_max_value_1':country_percent_difference_max_value_1,
             'country_percent_difference_max_value_2':country_percent_difference_max_value_2,'country_percent_difference_min':country_percent_difference_min,
             'country_percent_difference_min_date':country_percent_difference_min_date,'country_percent_difference_min_value_1':country_percent_difference_min_value_1,
             'country_percent_difference_min_value_2':country_percent_difference_min_value_2,
             'new_cases_jan_2020':country_new_cases_jan_2020, 'new_cases_feb_2020':country_new_cases_feb_2020,
             'new_cases_mar_2020':country_new_cases_mar_2020, 'new_cases_apr_2020':country_new_cases_apr_2020,
             'new_cases_may_2020':country_new_cases_may_2020, 'new_cases_june_2020':country_new_cases_june_2020,
             'new_cases_july_2020':country_new_cases_july_2020, 'new_cases_aug_2020':country_new_cases_aug_2020,
             'new_cases_sep_2020':country_new_cases_sep_2020, 'new_cases_oct_2020':country_new_cases_oct_2020,
             'new_cases_nov_2020':country_new_cases_nov_2020,'new_cases_dec_2020':country_new_cases_dec_2020})

In [517]:
country_name = dataset.location.unique()
df['country']=country_name

In [518]:
#Re-order the columns
columns_name = df.columns.to_list()
cols = []
cols.append(columns_name[-1])
cols.extend(columns_name[:-1])
df = df[cols]

In [519]:
#To view all of the columns
pd.set_option ('display.max_columns', None)

In [520]:
df

,country,country_last_total_case,country_first_case,country_first_case_date,country_last_case,country_last_case_date,country_percent_difference_max,country_percent_difference_max_date,country_percent_difference_max_value_1,country_percent_difference_max_value_2,country_percent_difference_min,country_percent_difference_min_date,country_percent_difference_min_value_1,country_percent_difference_min_value_2,new_cases_jan_2020,new_cases_feb_2020,new_cases_mar_2020,new_cases_apr_2020,new_cases_may_2020,new_cases_june_2020,new_cases_july_2020,new_cases_aug_2020,new_cases_sep_2020,new_cases_oct_2020,new_cases_nov_2020,new_cases_dec_2020
Afghanistan,Afghanistan,53332.0,1.0,2020-02-24,125.0,2021-01-08,35.000000,2020-08-02,1.0,36.0,-1.0,2020-02-24,1.0,0.0,0.0,1.0,174.0,1952.0,13081.0,16299.0,5158.0,1494.0,1109.0,2157.0,4849.0,5252.0
Albania,Albania,62378.0,2.0,2020-03-09,673.0,2021-01-08,14.000000,2020-05-19,1.0,15.0,-1.0,2020-12-31,589.0,0.0,0.0,0.0,243.0,530.0,364.0,1398.0,2741.0,4237.0,4136.0,7226.0,17307.0,20134.0
Algeria,Algeria,101657.0,1.0,2020-02-25,275.0,2021-01-08,15.333333,2020-03-20,3.0,49.0,-1.0,2020-02-25,1.0,0.0,0.0,1.0,715.0,3290.0,5388.0,4513.0,16487.0,14100.0,7036.0,6412.0,25257.0,16411.0
Andorra,Andorra,8489.0,1.0,2020-03-02,141.0,2021-01-08,78.000000,2020-06-01,1.0,79.0,-1.0,2020-03-02,1.0,0.0,0.0,0.0,376.0,369.0,19.0,91.0,70.0,251.0,874.0,2706.0,1989.0,1304.0
Angola,Angola,18066.0,1.0,2020-03-20,92.0,2021-01-08,10.000000,2020-07-11,4.0,44.0,-1.0,2020-03-21,1.0,0.0,0.0,0.0,7.0,20.0,59.0,198.0,864.0,1506.0,2318.0,5833.0,4334.0,2414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,Vietnam,1512.0,2.0,2020-01-23,3.0,2021-01-08,16.000000,2020-07-20,1.0,17.0,-1.0,2020-01-23,2.0,0.0,2.0,14.0,196.0,58.0,58.0,27.0,203.0,486.0,50.0,86.0,167.0,118.0
World,World,0.0,99.0,2020-01-23,0.0,2021-01-09,35.251196,2020-02-12,418.0,15153.0,-1.0,2021-01-08,812212.0,0.0,9372.0,76086.0,789726.0,2403476.0,2908473.0,4260585.0,7146706.0,7906979.0,8497564.0,12118910.0,17230604.0,19258662.0
Yemen,Yemen,2104.0,1.0,2020-04-10,0.0,2021-01-08,17.500000,2020-05-18,2.0,37.0,-11.0,2020-08-11,-1.0,10.0,0.0,0.0,0.0,6.0,317.0,835.0,570.0,230.0,76.0,29.0,128.0,-92.0
Zambia,Zambia,25326.0,2.0,2020-03-18,1029.0,2021-01-08,40.600000,2020-05-13,5.0,208.0,-1.0,2020-03-18,2.0,0.0,0.0,0.0,35.0,71.0,951.0,537.0,4369.0,6134.0,2662.0,1673.0,1215.0,3078.0


In [521]:
#Function to get value in dict
def get_key(mon,my_dict):
    for key, value in my_dict.items():
         if mon == key:
            return value
    return np.nan

In [522]:
month_name_number = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5,'june':6, 'july':7, 'aug':8, 'sep':9, 'oct':10,
                    'nov':11,'dec':12}

In [523]:
month_name_number.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [524]:
#Get the max and min % difference of new cases everyday in every country
country_percent_difference_max_monthly = {}
country_percent_difference_max_month = {}
country_percent_difference_max_value_1_monthly = {}
country_percent_difference_max_value_2_monthly = {}

country_percent_difference_min_monthly = {}
country_percent_difference_min_month = {}
country_percent_difference_min_value_1_monthly = {}
country_percent_difference_min_value_2_monthly = {}

for i in df.index.unique():
    data = df[(df.index)==i] ##IMPORTANT : THE INDEX STILL COUNTRY
    
    #Reset the index
    data['index']=[i for i in range(len(data))]
    data = data.set_index('index')
    
    max = 0
    min = 0
    
    for j in range(len(data)):
        for k in range(14,25):
            if (data.iloc[j,k])!=0:
                percent_difference = (data.iloc[j,k+1]-data.iloc[j,k]) / (data.iloc[j,k]) 
                if percent_difference > max:
                    max = percent_difference
                    country_percent_difference_max_value_1_monthly[i] = data.iloc[j,k]
                    country_percent_difference_max_value_2_monthly[i] = data.iloc[j,k+1]

                    mon=df.columns[k].split('_')[2]
                    country_percent_difference_max_month[i]=get_key(mon,month_name_number)
                    

                if percent_difference < min:
                    min = percent_difference
                    country_percent_difference_min_value_1_monthly[i] = data.iloc[j,k]
                    country_percent_difference_min_value_2_monthly[i] = data.iloc[j,k+1]

                    mon=df.columns[k].split('_')[2]
                    country_percent_difference_min_month[i]=get_key(mon,month_name_number)
            
    country_percent_difference_max_monthly[i]= max
    country_percent_difference_min_monthly[i]= min

<ipython-input-524-2b2d55824f09>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=[i for i in range(len(data))]


In [525]:
new = pd.DataFrame({'country_percent_difference_max_monthly':country_percent_difference_max_monthly,
                   'country_percent_difference_max_month':country_percent_difference_max_month,
                   'country_percent_difference_max_value_1_monthly':country_percent_difference_max_value_1_monthly,
                   'country_percent_difference_max_value_2_monthly':country_percent_difference_max_value_2_monthly,
                   'country_percent_difference_min_monthly':country_percent_difference_min_monthly,
                   'country_percent_difference_min_month':country_percent_difference_min_month,
                   'country_percent_difference_min_value_1_monthly':country_percent_difference_min_value_1_monthly,
                   'country_percent_difference_min_value_2_monthly':country_percent_difference_min_value_2_monthly})


In [526]:
df = pd.concat([df,new],axis=1, join='inner')

In [527]:
#Set index
df['index']=[i for i in range(len(df))]
df= df.set_index('index')
df

,country,country_last_total_case,country_first_case,country_first_case_date,country_last_case,country_last_case_date,country_percent_difference_max,country_percent_difference_max_date,country_percent_difference_max_value_1,country_percent_difference_max_value_2,country_percent_difference_min,country_percent_difference_min_date,country_percent_difference_min_value_1,country_percent_difference_min_value_2,new_cases_jan_2020,new_cases_feb_2020,new_cases_mar_2020,new_cases_apr_2020,new_cases_may_2020,new_cases_june_2020,new_cases_july_2020,new_cases_aug_2020,new_cases_sep_2020,new_cases_oct_2020,new_cases_nov_2020,new_cases_dec_2020,country_percent_difference_max_monthly,country_percent_difference_max_month,country_percent_difference_max_value_1_monthly,country_percent_difference_max_value_2_monthly,country_percent_difference_min_monthly,country_percent_difference_min_month,country_percent_difference_min_value_1_monthly,country_percent_difference_min_value_2_monthly
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Afghanistan,53332.0,1.0,2020-02-24,125.0,2021-01-08,35.000000,2020-08-02,1.0,36.0,-1.0,2020-02-24,1.0,0.0,0.0,1.0,174.0,1952.0,13081.0,16299.0,5158.0,1494.0,1109.0,2157.0,4849.0,5252.0,173.000000,2.0,1.0,174.0,-0.710353,7.0,5158.0,1494.0
1,Albania,62378.0,2.0,2020-03-09,673.0,2021-01-08,14.000000,2020-05-19,1.0,15.0,-1.0,2020-12-31,589.0,0.0,0.0,0.0,243.0,530.0,364.0,1398.0,2741.0,4237.0,4136.0,7226.0,17307.0,20134.0,2.840659,5.0,364.0,1398.0,-0.313208,4.0,530.0,364.0
2,Algeria,101657.0,1.0,2020-02-25,275.0,2021-01-08,15.333333,2020-03-20,3.0,49.0,-1.0,2020-02-25,1.0,0.0,0.0,1.0,715.0,3290.0,5388.0,4513.0,16487.0,14100.0,7036.0,6412.0,25257.0,16411.0,714.000000,2.0,1.0,715.0,-0.500993,8.0,14100.0,7036.0
3,Andorra,8489.0,1.0,2020-03-02,141.0,2021-01-08,78.000000,2020-06-01,1.0,79.0,-1.0,2020-03-02,1.0,0.0,0.0,0.0,376.0,369.0,19.0,91.0,70.0,251.0,874.0,2706.0,1989.0,1304.0,3.789474,5.0,19.0,91.0,-0.948509,4.0,369.0,19.0
4,Angola,18066.0,1.0,2020-03-20,92.0,2021-01-08,10.000000,2020-07-11,4.0,44.0,-1.0,2020-03-21,1.0,0.0,0.0,0.0,7.0,20.0,59.0,198.0,864.0,1506.0,2318.0,5833.0,4334.0,2414.0,3.363636,6.0,198.0,864.0,-0.443009,11.0,4334.0,2414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,1512.0,2.0,2020-01-23,3.0,2021-01-08,16.000000,2020-07-20,1.0,17.0,-1.0,2020-01-23,2.0,0.0,2.0,14.0,196.0,58.0,58.0,27.0,203.0,486.0,50.0,86.0,167.0,118.0,13.000000,2.0,14.0,196.0,-0.897119,8.0,486.0,50.0
188,World,0.0,99.0,2020-01-23,0.0,2021-01-09,35.251196,2020-02-12,418.0,15153.0,-1.0,2021-01-08,812212.0,0.0,9372.0,76086.0,789726.0,2403476.0,2908473.0,4260585.0,7146706.0,7906979.0,8497564.0,12118910.0,17230604.0,19258662.0,9.379386,2.0,76086.0,789726.0,0.000000,NaN,NaN,NaN
189,Yemen,2104.0,1.0,2020-04-10,0.0,2021-01-08,17.500000,2020-05-18,2.0,37.0,-11.0,2020-08-11,-1.0,10.0,0.0,0.0,0.0,6.0,317.0,835.0,570.0,230.0,76.0,29.0,128.0,-92.0,51.833333,4.0,6.0,317.0,-1.718750,11.0,128.0,-92.0


In [528]:
df.columns

Index(['country', 'country_last_total_case', 'country_first_case',
       'country_first_case_date', 'country_last_case',
       'country_last_case_date', 'country_percent_difference_max',
       'country_percent_difference_max_date',
       'country_percent_difference_max_value_1',
       'country_percent_difference_max_value_2',
       'country_percent_difference_min', 'country_percent_difference_min_date',
       'country_percent_difference_min_value_1',
       'country_percent_difference_min_value_2', 'new_cases_jan_2020',
       'new_cases_feb_2020', 'new_cases_mar_2020', 'new_cases_apr_2020',
       'new_cases_may_2020', 'new_cases_june_2020', 'new_cases_july_2020',
       'new_cases_aug_2020', 'new_cases_sep_2020', 'new_cases_oct_2020',
       'new_cases_nov_2020', 'new_cases_dec_2020',
       'country_percent_difference_max_monthly',
       'country_percent_difference_max_month',
       'country_percent_difference_max_value_1_monthly',
       'country_percent_difference_max_val

In [539]:
min = 0
for i in range(len(df)):
    if df.loc[i,'country_percent_difference_min_monthly']<min:
        min=df.loc[i,'country_percent_difference_min_monthly']
        country_min=df.loc[i,'country']

In [560]:
min_min = min
for i in range(len(df)):
    if df.loc[i,'country_percent_difference_min_monthly']>min_min and df.loc[i,'country_percent_difference_min_monthly']!=0:
        min_min=df.loc[i,'country_percent_difference_min_monthly']
        country_min_min=df.loc[i,'country']

In [569]:
#Search average
sum=0
for i in range(len(df)):
    sum+=df.loc[i,'country_percent_difference_min_monthly']
avg=sum/len(df)
avg

-0.803960715962731

In [573]:
#If min min
count_sum_min_min={}
for i in range(len(df)):
    count=0
    fv = df.loc[i,'country_last_total_case']
    while fv>10:
        fv = fv+(fv*min_min)
        count+=1
    count_sum_min_min[i]=count

In [574]:
#If min
count_sum_min={}
for i in range(len(df)):
    count=0
    fv = df.loc[i,'country_last_total_case']
    while fv>10:
        fv = fv+(fv*min)
        count+=1
    count_sum_min[i]=count

In [575]:
#If average
count_sum_avg={}
for i in range(len(df)):
    count=0
    fv = df.loc[i,'country_last_total_case']
    while fv>10:
        fv = fv+(fv*avg)
        count+=1
    count_sum_avg[i]=count

In [577]:
new_calc = pd.DataFrame({'count_sum_min_min':count_sum_min_min,'count_sum_min':count_sum_min,'count_sum_avg':count_sum_avg})

In [579]:
df = pd.concat([df,new_calc],axis=1, join='inner')

In [580]:
df

,country,country_last_total_case,country_first_case,country_first_case_date,country_last_case,country_last_case_date,country_percent_difference_max,country_percent_difference_max_date,country_percent_difference_max_value_1,country_percent_difference_max_value_2,country_percent_difference_min,country_percent_difference_min_date,country_percent_difference_min_value_1,country_percent_difference_min_value_2,new_cases_jan_2020,new_cases_feb_2020,new_cases_mar_2020,new_cases_apr_2020,new_cases_may_2020,new_cases_june_2020,new_cases_july_2020,new_cases_aug_2020,new_cases_sep_2020,new_cases_oct_2020,new_cases_nov_2020,new_cases_dec_2020,country_percent_difference_max_monthly,country_percent_difference_max_month,country_percent_difference_max_value_1_monthly,country_percent_difference_max_value_2_monthly,country_percent_difference_min_monthly,country_percent_difference_min_month,country_percent_difference_min_value_1_monthly,country_percent_difference_min_value_2_monthly,count_sum_min_min,count_sum_min,count_sum_avg
0,Afghanistan,53332.0,1.0,2020-02-24,125.0,2021-01-08,35.000000,2020-08-02,1.0,36.0,-1.0,2020-02-24,1.0,0.0,0.0,1.0,174.0,1952.0,13081.0,16299.0,5158.0,1494.0,1109.0,2157.0,4849.0,5252.0,173.000000,2.0,1.0,174.0,-0.710353,7.0,5158.0,1494.0,293,1,6
1,Albania,62378.0,2.0,2020-03-09,673.0,2021-01-08,14.000000,2020-05-19,1.0,15.0,-1.0,2020-12-31,589.0,0.0,0.0,0.0,243.0,530.0,364.0,1398.0,2741.0,4237.0,4136.0,7226.0,17307.0,20134.0,2.840659,5.0,364.0,1398.0,-0.313208,4.0,530.0,364.0,298,1,6
2,Algeria,101657.0,1.0,2020-02-25,275.0,2021-01-08,15.333333,2020-03-20,3.0,49.0,-1.0,2020-02-25,1.0,0.0,0.0,1.0,715.0,3290.0,5388.0,4513.0,16487.0,14100.0,7036.0,6412.0,25257.0,16411.0,714.000000,2.0,1.0,715.0,-0.500993,8.0,14100.0,7036.0,315,1,6
3,Andorra,8489.0,1.0,2020-03-02,141.0,2021-01-08,78.000000,2020-06-01,1.0,79.0,-1.0,2020-03-02,1.0,0.0,0.0,0.0,376.0,369.0,19.0,91.0,70.0,251.0,874.0,2706.0,1989.0,1304.0,3.789474,5.0,19.0,91.0,-0.948509,4.0,369.0,19.0,230,1,5
4,Angola,18066.0,1.0,2020-03-20,92.0,2021-01-08,10.000000,2020-07-11,4.0,44.0,-1.0,2020-03-21,1.0,0.0,0.0,0.0,7.0,20.0,59.0,198.0,864.0,1506.0,2318.0,5833.0,4334.0,2414.0,3.363636,6.0,198.0,864.0,-0.443009,11.0,4334.0,2414.0,256,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,1512.0,2.0,2020-01-23,3.0,2021-01-08,16.000000,2020-07-20,1.0,17.0,-1.0,2020-01-23,2.0,0.0,2.0,14.0,196.0,58.0,58.0,27.0,203.0,486.0,50.0,86.0,167.0,118.0,13.000000,2.0,14.0,196.0,-0.897119,8.0,486.0,50.0,171,1,4
188,World,0.0,99.0,2020-01-23,0.0,2021-01-09,35.251196,2020-02-12,418.0,15153.0,-1.0,2021-01-08,812212.0,0.0,9372.0,76086.0,789726.0,2403476.0,2908473.0,4260585.0,7146706.0,7906979.0,8497564.0,12118910.0,17230604.0,19258662.0,9.379386,2.0,76086.0,789726.0,0.000000,NaN,NaN,NaN,0,0,0
189,Yemen,2104.0,1.0,2020-04-10,0.0,2021-01-08,17.500000,2020-05-18,2.0,37.0,-11.0,2020-08-11,-1.0,10.0,0.0,0.0,0.0,6.0,317.0,835.0,570.0,230.0,76.0,29.0,128.0,-92.0,51.833333,4.0,6.0,317.0,-1.718750,11.0,128.0,-92.0,183,1,4
190,Zambia,25326.0,2.0,2020-03-18,1029.0,2021-01-08,40.600000,2020-05-13,5.0,208.0,-1.0,2020-03-18,2.0,0.0,0.0,0.0,35.0,71.0,951.0,537.0,4369.0,6134.0,2662.0,1673.0,1215.0,3078.0,12.394366,4.0,71.0,951.0,-0.566025,8.0,6134.0,2662.0,267,1,5


In [582]:
#Export to csv
covid_calc_csv_data = df.to_csv('covid_calc.csv', index = True) 